#**Reconhecimento de Padrões**

*Trabalho 4: PCA e Classificador SVM*

*- Kernel RBF*

*- Renoir Sampaio (389113)*

##**Preparação dos dados e informações**


In [1]:
# Importando bibliotecas
import numpy as np # Algebra Linear
from numpy.linalg import inv, det, norm # Inversa e determinante
import pandas as pd # Processamento de dados
import seaborn as sns # Plotagem
import matplotlib.pyplot as plt # Plotagem
import math # Matemática
from random import randint, uniform # Aleatoriedade
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.metrics import classification_report, confusion_matrix
!pip install qpsolvers
from qpsolvers import solve_qp

  Created wheel for qpsolvers: filename=qpsolvers-1.4-cp36-none-any.whl size=18455 sha256=39709f05b0783b5062e4368ba2a79d573c68e1960595158dd2bfbc237a35c260
  Stored in directory: /root/.cache/pip/wheels/ce/a7/5f/2af411a5eea3909096a303f84b7d6ff84e440081e8baccb4f2
  Created wheel for quadprog: filename=quadprog-0.1.7-cp36-cp36m-linux_x86_64.whl size=299426 sha256=b8337b2402b042860bd715361128bc5d5c8408af8005817a7486c413bf08e87f
  Stored in directory: /root/.cache/pip/wheels/36/dd/b1/849989444c0a5930927b260663019b7da6cff864fc224c2747
Successfully built qpsolvers quadprog


In [2]:
# Realiza a leitura do dataset
df = pd.read_csv('/content/TEJ - Normalized_2013_0430_Taiwan_data.csv', sep=',', encoding='latin-1', header=None)

In [3]:
# df = df.select_dtypes(include=['number'])
df = df.drop([0])
df = df.rename(columns = {0: 'classe'}, inplace = False)

In [4]:
# Selecionando atributos que serão usados
df = df.iloc[0:500, 0:94]

In [5]:
df = df.astype(float)
# df.info()

In [6]:
# Infomações sobre a base de dados
print(f'Número de amostras da base de dados: {df.shape[0]}')
print(f'Número de atributos da base de dados: {df.shape[1]-1}')
vetor_classes = np.unique(df['classe'])
print(f'Classes presentes na base de dados: {vetor_classes.tolist()}')

Número de amostras da base de dados: 500
Número de atributos da base de dados: 93
Classes presentes na base de dados: [0.0, 1.0]


Nota: Esta base de dados foram coletados do Taiwan Economic Journal para os anos de 1999 a 2009. A falência da empresa foi definida com base nos regulamentos de negócios da Bolsa de Valores de Taiwan.

In [7]:
# p Atributos (linhas) e N amostras (colunas)
df = df.T

In [8]:
# Mostrando o dataset
df

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500
classe,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.370594,0.464291,0.426071,0.399844,0.465022,0.388680,0.390923,0.508361,0.488519,0.495686,0.482475,0.444401,0.491152,0.474041,0.506703,0.513821,0.488909,0.535953,0.504071,0.487398,0.485253,0.504558,0.512017,0.494857,0.509969,0.444986,0.519280,0.547409,0.500853,0.416126,0.462195,0.476088,0.505874,0.486374,0.494369,0.475162,0.462926,0.478916,0.481061,0.486228,...,0.501487,0.438454,0.528104,0.497879,0.419100,0.528982,0.565203,0.515868,0.451421,0.446010,0.493248,0.448057,0.537074,0.537025,0.507483,0.451519,0.446887,0.511919,0.479501,0.472286,0.498562,0.480915,0.465363,0.467606,0.437332,0.469946,0.502559,0.493053,0.467021,0.497831,0.433676,0.494126,0.483401,0.485936,0.501926,0.495003,0.489251,0.496807,0.478721,0.493394
2,0.424389,0.538214,0.499019,0.451265,0.538432,0.415177,0.445704,0.570922,0.545137,0.550916,0.567543,0.549717,0.551570,0.533308,0.575829,0.571086,0.560238,0.590438,0.559802,0.543720,0.545573,0.564490,0.563672,0.548136,0.561382,0.503652,0.563618,0.593055,0.563945,0.470235,0.536034,0.544483,0.570704,0.544756,0.550916,0.564544,0.516354,0.541321,0.539468,0.548027,...,0.531454,0.478685,0.595835,0.553369,0.467292,0.582534,0.616550,0.558057,0.512211,0.505233,0.554187,0.488443,0.572503,0.562800,0.547263,0.508177,0.525240,0.578936,0.544429,0.536088,0.537178,0.539686,0.535543,0.536960,0.476232,0.536197,0.566071,0.590711,0.533744,0.562909,0.489097,0.547918,0.527911,0.563072,0.545464,0.573321,0.550698,0.561819,0.535597,0.542303
3,0.405750,0.516730,0.472295,0.457733,0.522298,0.419134,0.436158,0.559077,0.543284,0.542963,0.538198,0.498956,0.543391,0.523690,0.569838,0.558756,0.540286,0.580920,0.558649,0.533647,0.534665,0.553027,0.569035,0.540446,0.552599,0.495530,0.568606,0.592484,0.550565,0.463783,0.514428,0.529686,0.559827,0.540500,0.548584,0.525028,0.517105,0.529311,0.530864,0.544140,...,0.547031,0.488945,0.579314,0.545800,0.466513,0.572782,0.599925,0.571283,0.501579,0.490551,0.539483,0.498903,0.583918,0.568178,0.553188,0.505220,0.499920,0.566519,0.533701,0.525563,0.546603,0.534986,0.517908,0.518229,0.481343,0.521227,0.546657,0.546496,0.519193,0.547781,0.479522,0.549119,0.536860,0.540125,0.550083,0.543766,0.531667,0.549280,0.531399,0.540232
4,0.601457,0.610235,0.601450,0.583541,0.598783,0.590171,0.619950,0.601738,0.603612,0.599209,0.614026,0.623712,0.608131,0.600578,0.604686,0.621773,0.606524,0.618451,0.598344,0.636259,0.622177,0.607446,0.618934,0.609507,0.602956,0.611763,0.626544,0.609738,0.602027,0.599115,0.599987,0.618869,0.601169,0.603475,0.599108,0.614278,0.622703,0.607042,0.600881,0.598445,...,0.598178,0.592420,0.603417,0.599180,0.591944,0.607525,0.604318,0.600261,0.593645,0.591043,0.603749,0.593969,0.600455,0.621499,0.596448,0.600852,0.600254,0.601090,0.597479,0.605918,0.595526,0.597421,0.611777,0.617384,0.593278,0.616130,0.600686,0.600232,0.600520,0.601414,0.591281,0.600246,0.599216,0.600765,0.596081,0.598048,0.597220,0.604333,0.598481,0.595980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,.

In [9]:
# Divisão treino e teste
def divTesteTreinoTransp(df, percent, lengh):
  i = lengh*percent/100
  return df[0:int(i)], df[int(i):lengh]

In [10]:
# Criação de função para cálculo de acurácia baseado na coluna 'classe'  
# da tabela dados gerais, onde estão localizados os labels
def getAccuracy(testSet, predictions, tamanho):
    correct = 0
    for x in range(len(testSet)):
        if (testSet[x] == predictions[x]):
            correct += 1
    return (correct/float(len(testSet))) * 100.0

In [11]:
# Criar dataset aleatório
df_sample = df.T.sample(frac=1)
df_sample = df_sample.T

In [12]:
# Divisão treino e teste 70% e 30%
treino, teste = divTesteTreinoTransp(df_sample.T, 70, len(df.T))
treino = treino.T
teste = teste.T

In [13]:
# Mostrando dados de treino
treino

,325,33,482,49,355,310,477,302,490,158,311,240,53,229,328,76,189,434,100,59,418,280,198,424,154,252,289,497,488,221,272,299,13,224,118,408,318,414,73,214,...,409,117,136,276,485,70,338,381,321,15,326,266,416,217,169,210,257,222,58,82,166,187,464,449,436,290,390,422,24,34,378,423,244,16,476,278,314,341,126,260
classe,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.520645,0.505874,0.480915,0.493346,0.501341,0.495881,0.446887,0.462390,0.497831,0.480378,0.493492,0.434456,0.521913,0.491639,0.489056,0.489543,0.472286,0.457417,0.484278,0.482231,0.524692,0.504022,0.480232,0.488178,0.527617,0.451567,0.498221,0.489251,0.493053,0.526837,0.549651,0.449374,0.491152,0.507239,0.479452,0.500561,0.488666,0.453810,0.488812,0.517867,...,0.475991,0.485595,0.477210,0.491786,0.437332,0.496368,0.458051,0.517477,0.520840,0.506703,0.466046,0.506411,0.511822,0.484376,0.496222,0.488178,0.435041,0.487301,0.406669,0.432750,0.483742,0.465315,0.497879,0.364842,0.437820,0.479013,0.459123,0.442354,0.494857,0.486374,0.599132,0.478038,0.438844,0.513821,0.451519,0.473310,0.477843,0.465168,0.504753,0.471360
2,0.563781,0.570704,0.539686,0.550534,0.549826,0.542739,0.525240,0.464130,0.562909,0.546827,0.538923,0.481247,0.564871,0.542357,0.554241,0.547263,0.533035,0.523877,0.548190,0.544320,0.575720,0.565744,0.537069,0.535434,0.576047,0.494003,0.533580,0.550698,0.590711,0.578173,0.608482,0.505179,0.551570,0.555440,0.551952,0.549662,0.532599,0.513138,0.546391,0.568578,...,0.537505,0.570377,0.544865,0.555931,0.476232,0.549062,0.518153,0.580353,0.572830,0.575829,0.518916,0.566888,0.566997,0.561219,0.547536,0.538705,0.492205,0.551352,0.467292,0.492750,0.538269,0.526548,0.553369,0.318415,0.451483,0.530855,0.523550,0.533799,0.548136,0.544756,0.638247,0.559147,0.538705,0.571086,0.508177,0.525949,0.543066,0.541430,0.581880,0.538923
3,0.573211,0.559827,0.534986,0.539804,0.550725,0.543070,0.499920,0.516248,0.547781,0.533647,0.548798,0.498742,0.568178,0.539643,0.539162,0.535896,0.530114,0.509181,0.535575,0.536431,0.568125,0.557632,0.534022,0.540286,0.571765,0.502757,0.548316,0.531667,0.546496,0.580652,0.589432,0.500348,0.543391,0.557364,0.535253,0.553135,0.543177,0.506237,0.537770,0.568446,...,0.529579,0.541678,0.528080,0.543498,0.481343,0.545532,0.509877,0.571123,0.564324,0.569838,0.519942,0.555062,0.559345,0.535521,0.546014,0.542641,0.492585,0.539162,0.453450,0.482092,0.532951,0.511216,0.545800,0.393115,0.485465,0.532577,0.511162,0.489641,0.540446,0.540500,0.637240,0.528026,0.490658,0.558756,0.505220,0.525135,0.530971,0.515820,0.576102,0.536164
4,0.601630,0.601169,0.597421,0.610206,0.598279,0.600679,0.600254,0.593681,0.601414,0.600751,0.602899,0.596326,0.609291,0.600153,0.607597,0.607540,0.594531,0.606495,0.605183,0.618083,0.605954,0.608873,0.594423,0.597393,0.604953,0.594740,0.596297,0.597220,0.600232,0.605493,0.604484,0.589840,0.608131,0.600946,0.600369,0.603223,0.599872,0.595757,0.623986,0.610134,...,0.598402,0.603237,0.601976,0.600513,0.593278,0.612426,0.596499,0.597847,0.600643,0.604686,0.596167,0.605241,0.601306,0.602985,0.595195,0.602524,0.596715,0.599828,0.595994,0.604016,0.597890,0.604751,0.599180,0.494249,0.591706,0.596326,0.593659,0.595432,0.609507,0.603475,0.628396,0.603843,0.602618,0.621773,0.600852,0.596376,0.601551,0.600455,0.605133,0.595029
...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [14]:
# Mostrando dados de teste
teste

,98,465,141,32,162,171,429,142,365,296,473,334,330,322,489,269,262,413,188,84,239,67,339,467,37,344,426,291,207,71,6,372,304,90,397,170,265,446,347,301,...,236,357,274,404,47,238,362,350,167,26,312,7,402,398,102,233,130,66,69,45,283,110,163,405,393,124,499,300,480,456,199,57,275,340,180,352,452,208,370,8
classe,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.496807,0.419100,0.500122,0.476088,0.469166,0.498318,0.509336,0.567055,0.468630,0.490664,0.537074,0.480915,0.558768,0.455467,0.467021,0.481695,0.487203,0.470726,0.496319,0.480086,0.467508,0.473505,0.461463,0.565203,0.462926,0.475455,0.552674,0.454102,0.469751,0.544825,0.388680,0.506069,0.557939,0.468581,0.470238,0.496563,0.494711,0.485887,0.486521,0.478038,...,0.370107,0.499147,0.466777,0.452737,0.502121,0.486033,0.501292,0.481012,0.465315,0.444986,0.473358,0.390923,0.482913,0.507337,0.483401,0.345781,0.506508,0.475893,0.478331,0.508848,0.492517,0.446302,0.445522,0.498562,0.481353,0.492663,0.478721,0.490177,0.472286,0.478916,0.468093,0.066933,0.521962,0.453371,0.527470,0.484278,0.460001,0.492322,0.512748,0.508361
2,0.570704,0.467292,0.554405,0.544483,0.557185,0.562364,0.572667,0.595835,0.509976,0.565580,0.572503,0.543338,0.586731,0.499782,0.533744,0.515809,0.535925,0.507741,0.554132,0.541594,0.512538,0.514501,0.532109,0.616550,0.516354,0.539250,0.601232,0.516845,0.523332,0.588149,0.415177,0.559202,0.624618,0.529165,0.533962,0.559202,0.545901,0.535597,0.534998,0.527148,...,0.397187,0.543175,0.520770,0.491441,0.563509,0.537724,0.564544,0.528838,0.515645,0.503652,0.517608,0.445704,0.527911,0.552987,0.549608,0.393153,0.563836,0.540558,0.545901,0.553423,0.530909,0.531509,0.501254,0.557894,0.546664,0.550534,0.535597,0.532163,0.536088,0.420955,0.525240,0.057185,0.569560,0.515318,0.577791,0.538105,0.521151,0.547700,0.561655,0.570922
3,0.543552,0.466513,0.542106,0.529686,0.521013,0.549601,0.562932,0.602870,0.522887,0.545747,0.583918,0.536485,0.607474,0.507629,0.519193,0.535414,0.535093,0.523797,0.546389,0.534611,0.518979,0.527973,0.513678,0.599925,0.517105,0.526795,0.600996,0.505273,0.528508,0.589218,0.419134,0.565073,0.605653,0.522244,0.522833,0.546496,0.549173,0.543337,0.539911,0.530810,...,0.404144,0.544569,0.527330,0.501151,0.545854,0.534076,0.552224,0.535093,0.511591,0.495530,0.529472,0.436158,0.536592,0.562182,0.537609,0.386584,0.555008,0.528776,0.530275,0.552920,0.537663,0.499545,0.492532,0.548102,0.532363,0.542427,0.531399,0.540821,0.525563,0.525563,0.519246,0.054821,0.572086,0.503881,0.574067,0.533433,0.518336,0.543230,0.558863,0.559077
4,0.602848,0.591944,0.600455,0.618869,0.600383,0.605046,0.599447,0.604398,0.596910,0.604405,0.600455,0.598892,0.603324,0.603547,0.600520,0.606134,0.601573,0.594971,0.600852,0.597904,0.593133,0.595122,0.609450,0.604318,0.622703,0.598402,0.624786,0.597227,0.602906,0.600636,0.590171,0.599756,0.613702,0.598899,0.598286,0.602286,0.607842,0.595721,0.597378,0.599338,...,0.602351,0.599209,0.603626,0.598135,0.608866,0.597897,0.605363,0.596744,0.593782,0.611763,0.596758,0.619950,0.600701,0.599937,0.615928,0.586618,0.611914,0.600737,0.604102,0.635718,0.596470,0.595771,0.600643,0.598892,0.598373,0.604023,0.598481,0.600434,0.605918,0.598733,0.593566,0.601861,0.604686,0.596499,0.603799,0.597270,0.595144,0.604340,0.605198,0.601738
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [15]:
# Retirando classe do dataset de treino
treino_noclass = treino.T.drop(columns = ['classe'])
treino_noclass = treino_noclass.T

In [16]:
# Retirando classe do dataset de teste
teste_noclass = teste.T.drop(columns = ['classe'])
teste_noclass = teste_noclass.T

In [17]:
# Vetor média dos atributos de treino
treino_noclass_mean = treino_noclass.T.mean()

In [18]:
# Subtrair o valor médio de cada atributo da base de dados de treino
df_new = []
for i in range(len(treino_noclass)):
  df_prov = []
  for j in range(len(treino_noclass.T)):
    df_prov.append(treino_noclass.values[i][j] - treino_noclass_mean.values[i])
  df_new.append(df_prov)
df_new = pd.DataFrame(df_new)

In [19]:
# Matriz COV em função dos dados de treino
matrixCov_treino = pd.DataFrame.cov(df_new.T)

In [20]:
# Autovetores (U) autovalores (sigma)
sigma, U = np.linalg.eig(matrixCov_treino)

In [21]:
# Transformando 'U' em formato DataFrame
U = pd.DataFrame(U)

In [22]:
# Multiplicando qualquer coluna por ela mesma, valor esperado: 1
U[0].T @ U[0]

1.0000000000000002

In [23]:
# Transformando sigma em matriz diagonal
d_sigma = np.diag(sigma)

In [24]:
# Transformando matriz diagonal 'sigma' em formato DataFrame
d_sigma = pd.DataFrame(d_sigma)

In [25]:
# Somatório dos autovalores
sum_sigma = sigma.sum()

In [26]:
# Definindo q para dimenção da matriz de U transformada
# buscando 95% dos valores
for i in range(df_new.shape[0]):
  soma_prov = sigma[0:i].sum()
  porcent_sigma = soma_prov / sum_sigma * 100
  print(f'Com {i}: {int(porcent_sigma)}%')
  if porcent_sigma >= 95:
    q = i
    break
print(f'Sendo assim, a quantidade de colunas escolhidas foi {q}')

Com 0: 0%
Com 1: 46%
Com 2: 64%
Com 3: 74%
Com 4: 83%
Com 5: 88%
Com 6: 90%
Com 7: 92%
Com 8: 94%
Com 9: 95%
Sendo assim, a quantidade de colunas escolhidas foi 9


In [27]:
# Criar matriz transformada, valores descorrelacionados
matrix_transform = U.T @ df_new

In [28]:
# Matriz COV por multiplicação matricial com autovetores
# e dados de treino transformados
matrixCov_treino_transform = ((U @ matrixCov_treino) @ U.T)

In [29]:
# Matriz identidade por serem vetores ortogonais
matrizIdentidade = U.T @ U

In [30]:
# Decompondo matriz de teste
df_decomp = ((U @ d_sigma) @ U.T)

In [31]:
# Nova matriz de tranformação (que antes era o próprio U) será de dimenção pxq
# onde p ainda sera o número de atributos e q < p (definido previamente). 
# U de dimenção pxq transposto que multiplica df_new de dimenção pxN.
U_transf = U.iloc[:, 0:q]
matrix_transform_new = U_transf.T @ df_new

In [32]:
# Construindo o modelo para cada novo vetor dos dados de teste
df_teste = []
for i in range(len(teste_noclass)):
  df_prov = []
  for j in range(len(teste_noclass.T)):
    df_prov.append(teste_noclass.values[i][j] - treino_noclass_mean.values[i])
  df_teste.append(df_prov)
df_teste = pd.DataFrame(df_teste)
matrix_transform_teste = U_transf.T @ df_teste

## ***SVM (RBF):***

In [33]:
# 15 interações
all_accuracy_treino = []
all_accuracy_teste = []
melhor_acuracia = 0
melhor_C = 0
melhor_gama = 0
for p in range(15):
  print('#####################################################################')
  print(f'Interação: {p+1}/15')
  # Ordem do Kernel polinomial
  d = randint(15, 50)
  print(f'Valor de d = {d}')

  # Matriz X do treino
  X = matrix_transform_new

  # Vetor com as classes do treino
  treino_sample_class0 = treino.T.loc[treino.T['classe'] == 0]
  treino_sample_class1 = treino.T.loc[treino.T['classe'] == 1]
  y0 = -np.ones(treino_sample_class0.shape[0])
  y0 = pd.DataFrame(y0)
  y1 = np.ones(treino_sample_class1.shape[0])
  y1 = pd.DataFrame(y1)
  y = pd.concat([y0,y1])
  y_reset_index = y.reset_index(drop=True)

  # Cálculo do Kernel polinomial
  # Kernel RBF
  # Valor de Gama
  gama = uniform(0.001, 1000)
  K = pd.DataFrame(index=[i for i in range(X.shape[1])], columns=[i for i in range(X.shape[1])]) 
  for i in range(X.shape[1]):
      for j in range(X.shape[1]):
          K.iloc[i, j] = np.exp(-gama * (np.linalg.norm(X.iloc[:, i].T - X.iloc[:, j]) ** 2))

  # Formulação da programação quadrática
  C = uniform(0.001, 1000) # Constante de relaxamento
  a = np.zeros((X.shape[1], X.shape[1]), int)
  np.fill_diagonal(a, y_reset_index)
  y_diag = pd.DataFrame(a)
  P = ((y_diag @ K) @ y_diag)
  P = (P + P.T)/2
  P = np.array(P)
  P += np.identity(X.shape[1]) * 0.001
  P = P.astype('double')
  q = -C*np.ones(X.shape[1])
  G = -np.identity(X.shape[1])
  P = P.astype('double')
  h = np.zeros(X.shape[1])
  A = np.array(y_reset_index.T)[0]
  beq = np.array([0])

  # Chamada da função de otimização
  alpha = solve_qp(P, q, G, h, A, beq)

  # Achar vetores suporte
  vet_sup = np.argwhere(alpha > 1e-3)

  # Cálculo da constante b
  b = 0
  for i in vet_sup[:,0]:
    b = b + (1 / y_reset_index.iloc[i].values[0])
    for j in vet_sup[:,0]:
      b = b - (y_reset_index.iloc[j].values[0] * alpha[j] * K.iloc[j,i])
  b = b/vet_sup.shape[0]

  ############################ FASE DE TESTE ##################################
  
  # Matriz X do teste
  X = matrix_transform_teste

  # Vetor com as classes do teste
  teste_sample_class0 = teste.T.loc[teste.T['classe'] == 0]
  teste_sample_class1 = teste.T.loc[teste.T['classe'] == 1]
  y0 = -np.ones(teste_sample_class0.shape[0])
  y0 = pd.DataFrame(y0)
  y1 = np.ones(teste_sample_class1.shape[0])
  y1 = pd.DataFrame(y1)
  y = pd.concat([y0,y1])
  y_reset_index_teste = y.reset_index(drop=True)

  # Cálculo do Kernel polinomial
  # Kernel RBF
  # Valor de Gama
  gama = uniform(0.001, 1000)
  K = pd.DataFrame(index = [i for i in range(X.shape[1])], columns = [i for i in range(X.shape[1])]) 
  for i in range(X.shape[1]):
      for j in range(X.shape[1]):
          K.iloc[i, j] = np.exp(-gama * (np.linalg.norm(X.iloc[:, i].T - X.iloc[:, j]) ** 2))

  # Estimar classes
  aux = 0
  for j in vet_sup[:,0]:
    if j < K.shape[0]:
      aux = aux + (y_reset_index_teste.iloc[j].values[0] * alpha[j] * K.iloc[j,:])
  y_est_teste = np.sign((aux + b).T)
  y_est_teste = pd.DataFrame(y_est_teste)

  # Vetores de índices reais para comparação
  indices_teste = y_reset_index_teste.T.values[0]
  indices_teste = indices_teste.astype(float)

  # Vetores de índices previstos para comparação
  indices_previstos_teste = y_est_teste.T.values[0]
  indices_previstos_teste = indices_previstos_teste.astype(float)
  print(f'Matriz de confusão:\n{confusion_matrix(indices_teste, indices_previstos_teste)}')

  # Teste de acurácia com labels corretos e labels previstos (classificados)
  accuracy_teste = getAccuracy(indices_teste, indices_previstos_teste, len(indices_teste))
  all_accuracy_teste.append(accuracy_teste)
  print(f'Acurácia de teste dessa interação: {int(accuracy_teste)}%')
  print('#####################################################################')
  if p < 14:
    print('|/__\||/__\||/__\||/__\||/__\||/__\||/__\||/__\||/__\||/__\||/__\||/_')

  # Melhor acurácia e hiperparâmetros
  if accuracy_teste > melhor_acuracia:
    melhor_acuracia = accuracy_teste
    melhor_C = C
    melhor_gama = gama

print('\n')
# Média total das acurácias de teste
print(f'Média das acurácias de teste: {int(np.mean(all_accuracy_teste))}%')
# Melhor hiperparâmetro
print(f'No melhor valor de acurácia do teste, que foi igual a {int(melhor_acuracia)}%, o melhor valor para hiperparâmetro C foi {melhor_C} e o melhor para hiperparâmetro gama foi {melhor_gama}')

#####################################################################
Interação: 1/15
Valor de d = 27
Matriz de confusão:
[[141   0]
 [  0   9]]
Acurácia de teste dessa interação: 100%
#####################################################################
|/__\||/__\||/__\||/__\||/__\||/__\||/__\||/__\||/__\||/__\||/__\||/_
#####################################################################
Interação: 2/15
Valor de d = 49
Matriz de confusão:
[[141   0]
 [  0   9]]
Acurácia de teste dessa interação: 100%
#####################################################################
|/__\||/__\||/__\||/__\||/__\||/__\||/__\||/__\||/__\||/__\||/__\||/_
#####################################################################
Interação: 3/15
Valor de d = 44
Matriz de confusão:
[[141   0]
 [  4   5]]
Acurácia de teste dessa interação: 97%
#####################################################################
|/__\||/__\||/__\||/__\||/__\||/__\||/__\||/__\||/__\||/__\||/__\||/_
##########################

Nota: O kernel RBF consegue uma ótima classificação. Tendo média geral de acurácia do teste de 99% e ótimos valores para recall e precisão também, já que as classes conseguem ser bem separadas sem nenhum erro de classificação em quase todos os casos.